In [1]:
import os
import sys
import pygsheets
import math
import numpy as np
import pandas as pd
import ams_utilities as au
from openpyxl import load_workbook
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import AffinityPropagation
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from itertools import cycle
import scipy
%matplotlib notebook
import pyabf
import paq2py
import scipy.interpolate
import matplotlib.pyplot as plt
import scipy.stats as ss 
from itertools import product

In [2]:
import pygsheets
from morph_analysis_jake import *
directory = r"\\163.1.248.177\Root\jswann\Morphologies"

In [3]:
#list datapath sheets
datapaths = []
for root, subdirs, files in os.walk(directory):
    for filename in files:
        if filename.endswith(".xlsx"):
            temppath=os.path.join(root, filename)
            datapaths.append(temppath)
print(len(datapaths))

59


In [4]:
#create dataframe
df_all = {}
for i in datapaths:

    df_single_cell = make_param_df(i)

    if i == datapaths[0]:
        df_all = df_single_cell

    else:

        df_all = pd.concat([df_all, df_single_cell], axis=1, join='outer', join_axes=None, ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, sort=None, copy=True)

#Sort columns into order
df_all=df_all.reindex(sorted(df_all.columns),axis=1)

C:\Users\Packer Lab\Documents\Python Scripts\Code\morph_analysis_jake.py:79: RuntimeWarning: invalid value encountered in double_scalars
  axon_len_SA_ratio = axon_length_total/axon_SA
C:\Users\Packer Lab\Documents\Python Scripts\Code\morph_analysis_jake.py:361: RuntimeWarning: invalid value encountered in double_scalars
  dendrite_len_SA_ratio = dendrite_length_total/dendrite_SA


In [5]:
df_all.head()

,AS016: Cell01,AS016: Cell02,AS029: Cell01,AS030: Cell01,AS035: Cell01,AS036: Cell01,AS038: Cell01,AS039: Cell01,AS039: Cell02,AS040: Cell01,...,AS114: Cell01,AS114: Cell02,AS114: Cell03,AS115: Cell01,AS116: Cell01,AS116: Cell02,AS117: Cell04,AS132: Cell03,AS143: Cell03,AS152: Cell01
Axon_node_total,6.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,5.000000,6.000000,6.000000,0.000000
Axon_length,1620.490000,0.0,28.400000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,118.660000,0.0,0.0,0.0,0.0,0.0,355.680000,1137.770000,629.870000,123.850000
Axon_SA,10859.800000,0.0,104.620000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,454.330000,0.0,0.0,0.0,0.0,0.0,981.420000,5585.990000,2131.560000,189.150000
Axon_length_to_SA_ratio,0.149219,NaN,0.271459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.261176,NaN,NaN,NaN,NaN,NaN,0.362414,0.203683,0.295497,0.654771
Highest_order_axon_segment,4.000000,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,NaN,NaN,NaN,NaN,NaN,4.000000,7.000000,7.000000,1.000000


In [6]:
#add slice orientations
if 'Slice_Orientation' not in df_all.index:
    text_file=open('E:\\Jake\\Orientations.txt')
    orientations=list(text_file.read())
    orientations=pd.Series(orientations).to_frame().T
    orientations.index=['Slice_Orientation']
    orientations.columns=df_all.columns
    df_all=pd.concat([orientations,df_all], join='inner')

In [7]:
df_all.head()

,AS016: Cell01,AS016: Cell02,AS029: Cell01,AS030: Cell01,AS035: Cell01,AS036: Cell01,AS038: Cell01,AS039: Cell01,AS039: Cell02,AS040: Cell01,...,AS114: Cell01,AS114: Cell02,AS114: Cell03,AS115: Cell01,AS116: Cell01,AS116: Cell02,AS117: Cell04,AS132: Cell03,AS143: Cell03,AS152: Cell01
Slice_Orientation,H,H,H,H,H,C,H,H,H,H,...,C,C,C,C,C,C,C,C,C,C
Axon_node_total,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,5,6,6,0
Axon_length,1620.49,0,28.4,0,0,0,0,0,0,0,...,118.66,0,0,0,0,0,355.68,1137.77,629.87,123.85
Axon_SA,10859.8,0,104.62,0,0,0,0,0,0,0,...,454.33,0,0,0,0,0,981.42,5585.99,2131.56,189.15
Axon_length_to_SA_ratio,0.149219,NaN,0.271459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.261176,NaN,NaN,NaN,NaN,NaN,0.362414,0.203683,0.295497,0.654771


In [8]:
#Add polar analysis data
if 'Dendrite_polar_length_mean' not in df_all.index:
    df_polar=pd.read_excel('E:\\Jake\\2020-05-31_Morphologies_PolarMetrics.xlsx', indexCol=0)
    df_polar=df_polar.drop(['Unnamed: 0'],axis=1)
    variableNames=['Dendrite_polar_length_mean', 
             'Dendrite_polar_length_std',
             'Dendrite_polar_length_variance',
             'Dendrite_polar_longest_bin']
    df_polar.index=variableNames
    df_all=pd.concat([df_all,df_polar], join='inner') #This will drop any cells with no dendritic data
df_polar.T

,Dendrite_polar_length_mean,Dendrite_polar_length_std,Dendrite_polar_length_variance,Dendrite_polar_longest_bin
AS016: Cell01,0.398822,0.211449,0.044711,260.0
AS016: Cell02,0.371481,0.236290,0.055833,140.0
AS029: Cell01,0.374605,0.300087,0.090052,340.0
AS030: Cell01,0.327464,0.236692,0.056023,100.0
AS035: Cell01,0.321695,0.279975,0.078386,340.0
AS036: Cell01,0.349470,0.310290,0.096280,260.0
AS038: Cell01,0.459234,0.181563,0.032965,100.0
AS039: Cell01,0.379131,0.255947,0.065509,340.0
AS039: Cell02,0.638801,0.186113,0.034638,300.0
AS040: Cell01,0.390497,0.241370,0.058259,340.0


In [9]:
#export to google sheet
creds = r"C:\Users\Packer Lab\Documents\Python Scripts\Code\creds_jake.json"
gc = pygsheets.authorize(service_file=creds)

df = pd.DataFrame()

params = list(df_all.index.values)
df['Parameters'] = params
df.columns = ['Mouse#: Cell']

sh = gc.open('Morph_analysis')

wks = sh[0]
wks.set_dataframe(df, (1,1))
wks.set_dataframe(df_all, (1,2))